# Generating Synthetic Data 
Using OpenAI's API GPT-4 
____

In [63]:
import openai
import random
import os
import dotenv
import llm
import pandas as pd
import re

In [ ]:
key = os.environ.get('OPENAI_API_KEY')

### LLM to create a dataset with a same prompt-response as app.py
A transition from the gpt-4 API to a local llm will require fine-tuning to uphold performance with a smaller model. To fine-tune to this specific use-case, the training data needs to resemble the prompt-response pair found when interacting with app.py. The formatting is quite extensive:

### Prompt Engineering
Leveraging how capable gpt-4 is by explicitly spelling out the expected prompt and response input outputs. 

In [43]:
prompt_definition = """A model that receives an instruction from the user to show certain information from a postgres database in colloquial english. 
Followed by information on table definitions under TABLE_DEFINITIONS formatted as the output of this function:

    def get_table_definitions_for_prompt(self):
        table_names = self.get_all_table_names()
        return "\n".join([self.get_table_definition(table) for table in table_names])

With the instruction and the table definitions formatted like so:

<user instruction>

TABLE_DEFINITIONS

<output of function> """

response_definition = """

It is the model's job to then respond with an SQL query that will be run to fetch the requested data with an output that follows this formatting:

<insert an explanation of the sql query as raw text here>
            ---------
<insert sql query exclusively as raw text here>

Make sure to include the --------- in the answer.
"""

prompt = f'{prompt_definition}{response_definition}'

In [ ]:
#configuring the gpt-4 model params
#reasonable temperature to promote some creativity
#number_of_examples cannot exceed 15 as that reaches token limit provided by openai API
temperature = 0.4
number_of_examples = 15

In [22]:
#messages will describe the role of the llm model to generate synthetic data
def generate_example(prompt, prev_examples, temperature=.5):
    
    #system message will set the role of the llm
    messages=[
        {
            "role": "system",
            "content": f"""You are generating data which will be used to train a machine learning model.\n\n
            You will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\n
            You will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"""
        }
    ]

#storing previous examples into 'messages' list so that the model can avoid repetitions, ensure diversity of the outputs.
    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })



#configuring the gpt-4 model
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

In [23]:
#storing generated prompt-pairs into list
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
["prompt\n-----------\nShow me the names of all the employees.\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text)\n-----------\n\nresponse\n-----------\nTo get the names of all the employees, we will query the 'Employees' table and select the 'name' column.\n\n---------\nSELECT name FROM Employees;\n-----------", "prompt\n-----------\nI need to see the names and ages of all employees.\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text)\n-----------\n\nresponse\n-----------\nTo fetch the n

In [24]:
# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

pd.set_option('display.max_colwidth', None)
df.head()

,prompt,response
0,Show me the names of all the employees.\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text),"To get the names of all the employees, we will query the 'Employees' table and select the 'name' column.\n\n---------\nSELECT name FROM Employees;"
1,I need to see the names and ages of all employees.\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text),"To fetch the names and ages of all employees, we need to select the 'name' and 'age' columns from the 'Employees' table.\n\n---------\nSELECT name, age FROM Employees;"
2,Can you show me the names of all the departments?\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text),"To get the names of all the departments, we will query the 'Departments' table and select the 'name' column.\n\n---------\nSELECT name FROM Departments;"
3,I'd like to see the names of all employees who are over 30 years old.\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text),"To fetch the names of all employees who are over 30 years old, we need to select the 'name' column from the 'Employees' table where the 'age' is greater than 30.\n\n---------\nSELECT name FROM Employees WHERE age > 30;"
4,Show me the names of all employees in the 'Marketing' department.\n\nTABLE_DEFINITIONS\n\nEmployees:\nid (integer)\nname (text)\nage (integer)\ndepartment_id (integer)\n\nDepartments:\nid (integer)\nname (text),"To get the names of all employees in the 'Marketing' department, we need to join the 'Employees' and 'Departments' tables on the 'id' and 'department_id' columns respectively. Then, we select the 'name' column where the department 'name' is 'Marketing'.\n\n---------\nSELECT Employees.name FROM Employees JOIN Departments ON Employees.department_id = Departments.id WHERE Departments.name = 'Marketing';"


In [25]:
# Split the data into train and test sets, with 90% in the train set
# train_df = df.sample(frac=0.9, random_state=42)
# test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
# train_df.to_csv('train_df.csv', index = False)
df.to_csv('df4.csv', index = False)
# test_df.to_csv('test.csv', index=False)

In [28]:
df1 = pd.read_csv('./data/df1.csv')
df2 = pd.read_csv('./data/df2.csv')
df3 = pd.read_csv('./data/df3.csv')
df4 = pd.read_csv('./data/df4.csv')

In [35]:
def join_df(*dataframes):
    joined_df = pd.concat(dataframes, ignore_index = True)
    return joined_df

df = join_df(df1,df2,df3,df4)

df.to_csv('sql_prompt_pairs.csv',index = False)

In [49]:
df = pd.read_csv('./data/sql_prompt_pairs.csv')

In [64]:
#storing the prompts for future evaluation step
pattern = r'[.?\n]'
prompts = []
for i in range(len(df)):
    prompt_sentence = re.split(pattern, df.loc[i, 'prompt'])[0]
    prompts.append(prompt_sentence)


In [69]:
prompts = pd.Series(prompts)
prompts.to_csv('./data/prompts.csv', index = False)